In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy.sparse import csc_matrix, csr_matrix, hstack
import time
import numpy as np
import pickle

from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization,CuDNNLSTM,Activation,BatchNormalization,Conv2D
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D,GlobalAveragePooling2D, GlobalMaxPooling1D,GlobalMaxPooling2D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.utils import shuffle
from keras.preprocessing import text, sequence
from sklearn.metrics import log_loss,roc_auc_score
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import l1_l2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
import re
from keras import backend as K
# To get learning rate
from sklearn.preprocessing import QuantileTransformer
from keras.layers import Dot, multiply
from keras.layers import Reshape


/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_pickle(FILE.train_ori.value)
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle(FILE.test_ori.value)
print('test shape is: {}'.format(test.shape))

# X = pd.concat([train.drop(['click'],axis=1),test])
X = pd.concat([train,test],sort=False)
print(X.shape)

X_clean = pd.read_csv('../../data/original/cleaned_data_price_final.csv')
X_wei = pd.read_pickle('../../data/nn_features/clean.pkl')

X_shiyi = pd.read_pickle(FILE.shiyi_fillna_ori.value)
print(X_shiyi.shape)

X = X.merge(X_shiyi[['time_hour','instance_id']],how='inner',on='instance_id')
print(X.shape)
assert np.sum(X_clean['instance_id'].values != X['instance_id'].values) == 0
assert np.sum(X_wei['instance_id'].values != X['instance_id'].values) == 0
X['model'] = X_wei['model_new'].values
# X.drop('model',axis=1,inplace=True)
# X['price'] = X_clean['price'].values
# X['city_province'] = X_wei['city_province'].values
# X['province'] = X_wei['province1'].values
# X.drop(['city','province'],axis=1,inplace=True)
# X['make'] = X_wei['make_new'].values

ignore_columns = ['instance_id','time','click'] + ['creative_is_js', 'creative_is_voicead', 'app_paid']
X_nu = X_clean[['price']].copy()
X_nu['time'] = (X['time'] - X['time'].min()).values

train shape is: (1001650, 35)
test shape is: (40024, 34)
(1041674, 35)
(1041674, 45)
(1041674, 36)


In [3]:
doc_col=['user_tags']
numu_col=[]
non_doc_col = list(set(X.columns) - set(ignore_columns) - set(doc_col))
def combALL(X,sep=' '):
    le = LabelEncoder()
    ret = pd.DataFrame()
    cols = list(X.columns)
    if cols[0] in ['city','province']:
        ret['comALL'] = 'cp_'+X[cols[0]].astype(str)
    else:
        ret['comALL'] = le.fit_transform(X[cols[0]].astype(str))
        ret['comALL'] = cols[0]+'_'+ret['comALL'].astype(str)
    for col in tqdm(cols[1:]):
        if col in ['city','province']:
            ret['tmp'] = 'cp_'+X[col].astype(str)
            ret['comALL'] = ret['comALL'] + sep + ret['tmp'].astype(str)
        else:
            ret['tmp'] = le.fit_transform(X[col].astype(str))
            ret['comALL'] = ret['comALL'] + sep + col + '_'+ ret['tmp'].astype(str)
    return ret['comALL'].values

In [4]:
X['comALL'] = combALL(X[non_doc_col])

100%|██████████| 28/28 [00:58<00:00,  2.08s/it]


In [5]:
need_process_col = list(set(X.columns) - set(ignore_columns))


doc_col = ['user_tags']
non_doc_col = [f for f in need_process_col if f not in doc_col]
non_doc_col = ['comALL']
X_ = X[non_doc_col].copy()

X_doc = X[doc_col].copy()

    
    
for col in tqdm(doc_col):
    X_doc[col] = X_doc[col].astype(str)
    
    

    

100%|██████████| 1/1 [00:00<00:00,  4.32it/s]


In [6]:
X_.iloc[0].values

array(['creative_is_jump_1 os_name_0 creative_type_4 app_id_232 app_cate_id_7 creative_tp_dnf_13 carrier_1 advert_id_1 devtype_2 creative_is_download_0 adid_553 inner_slot_id_623 creative_width_2 osv_153 cp_137103102105100 cp_137103102100100 orderid_302 creative_id_209 f_channel_77 model_2269 os_2 creative_has_deeplink_0 time_hour_12 advert_industry_inner_17 creative_height_9 nnt_1 advert_name_25 campaign_id_1 make_864'],
      dtype=object)

In [7]:
# # def comb_fe(X,cols,sep=' '):
# #     ret = X[cols[0]].astype(str).copy()
# #     for col in cols[1:]:
# #         ret = ret + sep + X[col].astype(str)
# #     return ret.values

# def process_slot(x):
#     x = re.sub(r'[-_]',' ',x)
#     x = x.split(' ')
#     ret = ['p{}_'.format(c)+x[c] for c in range(len(x))]
#     ret = ' '.join(ret)
#     return ret
        
    

# def comb_fe(X,cols,sep=' '):
#     def add_col_name(x,colName,splitter=' ' ,sep=' '):
#         ret = [colName+'_'+each for each in x.split(splitter)]
#         return sep.join(ret)
#     ret = pd.DataFrame()
#     if cols[0] == 'user_tags':
#         ret['comb'] = X['user_tags'].astype(str).apply(add_col_name,colName='userTags',splitter=',',sep=sep)
#     else:
#         if cols[0] in original_name_col:
#             ret['comb'] = X[cols[0]].astype(str).copy()
#         else:
#             ret['comb'] = X[cols[0]].astype(str).apply(add_col_name,colName=cols[0],splitter=' ',sep=sep)

#     for col in tqdm(cols[1:]):
#         if col == 'user_tags':
#             ret['new_feature'] = X['user_tags'].astype(str).apply(add_col_name,colName='userTags',splitter=',',sep=sep)
#         else:
#             if col in original_name_col:
#                 ret['new_feature'] = X[col].astype(str).copy()
#             else:
#                 ret['new_feature'] = X[col].astype(str).apply(add_col_name,colName=col,splitter=' ',sep=sep)
#         ret['comb'] =( ret['comb'] + sep + ret['new_feature']).values
#     return ret['comb'].values

# processed_col = []
# original_name_col = ['model','make','os','osv']

# doc_col=['user_tags']
# X_doc = X[doc_col].copy()
# processed_col.extend(doc_col)
# cob_col = []
# for each in cob_col:
#     feature_name = '_'.join(each)
#     processed_col.extend(each)
#     doc_col.append(feature_name)
#     X_doc[feature_name] = comb_fe(X,each)
    
# non_doc_col = list(set(X.columns) - set(ignore_columns) - set(processed_col))
# non_doc_col = ['comALL']
# #!!!!!!!!!!!!!!!!!!!!!!!!
# # 0.414797
# # non_doc_col = list(set(X.columns) - set(ignore_columns)- set(doc_col))
# X_ = X[non_doc_col].copy()

# # for col in tqdm(non_doc_col):
# #     X_[col] = le.fit_transform(X_[col].astype(str))
# #     X_[col] = col + '_'+X_[col].astype(str)
    
# for col in tqdm(doc_col):
#     if col=='inner_slot_id':
#         X_doc[col] = X_doc[col].astype(str).apply(process_slot)
#     else:
#         X_doc[col] = X_doc[col].astype(str)
# numu_col= list(X_nu.columns)
# numu_col = []
        

In [8]:
train_index = pickle.load(open(FILE.train_index.value,'rb'))
holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))

In [9]:
num_folds = 7
seed = 1001
train_index_list = []
val_index_list = []
folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
for t,v in folds.split(train.loc[train_index],train.loc[train_index,'click']):
    train_index_list.append(train.loc[train_index].iloc[t].index)
    val_index_list.append(train.loc[train_index].iloc[v].index)
    
check = []
for i in val_index_list:
    check.extend(list(i))
assert len(set(check+list(holdout_index))) == len(train)

In [10]:
train_fold_y = {}
val_fold_y = {}
holdout_y = train.loc[holdout_index,'click'].values
for fold in range(num_folds):
    train_fold_y[fold] = train.loc[train_index_list[fold],'click'].values
    val_fold_y[fold] = train.loc[val_index_list[fold],'click'].values

In [11]:
info_dict = {}
train_all_dict = {}
train_fold_dict = {}

val_fold_dict = {}

holdout_input_dict = {}
test_input_dict = {}
maxlen = 1
prefix_input_nonDoc = 'input_'
prefix_input_Doc = 'input_rnn_'
prefix_input_nu = 'input_Num_'
sequence_size_dict_nonDoc = {}
for col in tqdm(non_doc_col):
    
    maxlen = 1
    if col in ['city','province']:
        maxlen = 5
        filters = ' '
    elif col == 'model':
        maxlen = 1
        filters = '@'
    else:
        maxlen = 50
        filters=' '
    sequence_size_dict_nonDoc[col]=maxlen
    tok=text.Tokenizer(num_words=X_[col].nunique(),lower=False,filters=filters)
    tok.fit_on_texts(list(X_[col]))
    info_dict.update({prefix_input_nonDoc+col:{'tok':tok}})
    t = tok.texts_to_sequences(list(X_[col].iloc[:1001650].values))
    te = tok.texts_to_sequences(list(X_[col].iloc[1001650:].values))
    train_all_dict[prefix_input_nonDoc+col] = sequence.pad_sequences(t,maxlen=maxlen)
    test_input_dict[prefix_input_nonDoc+col] = sequence.pad_sequences(te,maxlen=maxlen)
    holdout_input_dict[prefix_input_nonDoc+col] = train_all_dict[prefix_input_nonDoc+col][list(holdout_index)]
    
    for fold in range(num_folds):
        if train_fold_dict.get(fold) is None:
            train_fold_dict[fold] = {}
            val_fold_dict[fold] = {}
        train_fold_dict[fold].update({prefix_input_nonDoc+col: train_all_dict[prefix_input_nonDoc+col][list(train_index_list[fold])]})
        val_fold_dict[fold].update({prefix_input_nonDoc+col:train_all_dict[prefix_input_nonDoc+col][list(val_index_list[fold])]})
        
sequence_size_dict = {}
for col in tqdm(doc_col):
    if 'user_tags' in col:
        maxlen = 100
        tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False,filters=',')
    else:
        maxlen = 20
        tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False,filters=' ')
    tok.fit_on_texts(list(X_doc[col]))
    info_dict.update({prefix_input_Doc+col:{'tok':tok}})
    sequence_size_dict[col] = maxlen
    t = tok.texts_to_sequences(list(X_doc[col].iloc[:1001650].values))
    te = tok.texts_to_sequences(list(X_doc[col].iloc[1001650:].values))
    train_all_dict[prefix_input_Doc+col] = sequence.pad_sequences(t,maxlen=maxlen)
    test_input_dict[prefix_input_Doc+col] = sequence.pad_sequences(te,maxlen=maxlen)
    holdout_input_dict[prefix_input_Doc+col] = train_all_dict[prefix_input_Doc+col][list(holdout_index)]
    
    for fold in range(num_folds):
        if train_fold_dict.get(fold) is None:
            train_fold_dict[fold] = {}
            val_fold_dict[fold] = {}
        train_fold_dict[fold].update({prefix_input_Doc+col: train_all_dict[prefix_input_Doc+col][list(train_index_list[fold])]})
        val_fold_dict[fold].update({prefix_input_Doc+col:train_all_dict[prefix_input_Doc+col][list(val_index_list[fold])]})
        
print('process numerical input')
train_all_dict[prefix_input_nu] = X_nu[numu_col].iloc[:1001650].values
test_input_dict[prefix_input_nu] = X_nu[numu_col].iloc[1001650:].values
holdout_input_dict[prefix_input_nu] = train_all_dict[prefix_input_nu][list(holdout_index)]
for fold in range(num_folds):
        if train_fold_dict.get(fold) is None:
            train_fold_dict[fold] = {}
            val_fold_dict[fold] = {}
        train_fold_dict[fold].update({prefix_input_nu: train_all_dict[prefix_input_nu][list(train_index_list[fold])]})
        val_fold_dict[fold].update({prefix_input_nu:train_all_dict[prefix_input_nu][list(val_index_list[fold])]})

100%|██████████| 1/1 [00:27<00:00, 27.24s/it]


process numerical input


# Build NN model only use model

# 0.41456-- 71 epoch-- include price

In [25]:

def get_nn_model(cols,doc_cols=[],numu_cols=[]):
    """
    cols, used to do ebd and dense layers
    doc_cols: used to do rnn
    there can be overlaps
    """
    input_list = []
    concat_list = []
    doc_list = []
    numu_list = []
    for col in cols:
        max_feature = len(info_dict[prefix_input_nonDoc+col]['tok'].index_word)
        embed_size = int(np.log2(max_feature)/np.log2(1.5))
        print('non doc ebd size:{}'.format(embed_size))
        if embed_size< 2:
            embed_size = 2
        input_shape = sequence_size_dict_nonDoc[col]
        cur_input = Input(shape=(input_shape, ),name = prefix_input_nonDoc+col)
        
       
        embed_layer = Embedding(max_feature,
                            embed_size,
#                                 100,
                            input_length=input_shape,
                            trainable=True,
#                                 embeddings_initializer='he_normal',
                            embeddings_regularizer=l2(0.0005),
                            name='ebd_'+col)(cur_input)
        embed_layer = SpatialDropout1D(0.5)(embed_layer)
        x = Reshape((int(embed_layer.shape[1]),
                     int(embed_layer.shape[2]),
                     1),name='reshape')(embed_layer)
        x = Flatten()(embed_layer)
        input_list.append(cur_input)
        concat_list.append(x)
    for col in doc_cols:
        max_feature = len(info_dict[prefix_input_Doc+col]['tok'].index_word)
        embed_size = int(np.log2(max_feature)/np.log2(1.5))
        if embed_size< 2:
            embed_size = 2
        input_shape = sequence_size_dict[col]
        cur_input = Input(shape=(input_shape, ),name = prefix_input_Doc+col)
        embed_layer = Embedding(max_feature,
                            embed_size,
#                                 embeddings_initializer='he_normal',
                            input_length=input_shape,
                            trainable=True,
                            embeddings_regularizer=l2(0.0005),
                            name='ebd_rnn_'+col)(cur_input)
        x = SpatialDropout1D(0.5)(embed_layer)
        x = Bidirectional(CuDNNGRU(25, return_sequences=True))(x)
#         x = Reshape((int(x.shape[1]),
#                      int(x.shape[2]),
#                      1),name='reshape')(x)
#         x1 = Conv2D(filters=64,kernel_size=(3,5))(x)
#         x_aveP1 = GlobalAveragePooling2D()(x1)
#         x_maxP1 = GlobalMaxPooling2D()(x1)
#         x = concatenate([x_aveP1,x_maxP1])
#         x = Flatten()(x)
        x = Conv1D(25, kernel_size = 3, padding = "valid", kernel_initializer = "he_normal")(x)
        x_aveP = GlobalAveragePooling1D()(x)
        x_maxP = GlobalMaxPooling1D()(x)
        x = concatenate([x_aveP,x_maxP])
        input_list.append(cur_input)
        concat_list.append(x)

    if len(numu_cols) > 0:
        print('add numu...')
        nu_shape = len(numu_cols)
        cur_input = Input(shape=(nu_shape, ),name = prefix_input_nu)
        x = BatchNormalization()(cur_input)
        x = Dense(1, activation=None)(x)
#         x = Dropout(0.2)(x)
        input_list.append(cur_input)
        numu_list.append(x)
       
    if len(concat_list) > 1:
        x = concatenate(concat_list)
#     x = BatchNormalization()(x)
    

    if len(numu_list)>0:
        x = concatenate([x]+numu_list)
#         x = BatchNormalization()(x)
        
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)

    preds = Dense(1, activation="sigmoid")(x)
    model = Model(input_list, preds)
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])    
    return model

In [13]:
def train_each_fold(input_train_dict,input_val_dict,y_train,y_val,cols,doc_col=[],numu_col=[],tolerance=30,):
    model = get_nn_model(cols,doc_col,numu_col)
    cur_to = 0
    best_logloss = None
    best_weights = None
    count = 0
    base_lr = 0.001
    train_loss_list = []
    val_loss_list = []
    while True:
#         decay = (80 - count)/80
#         decay = max(decay,0.1)
#         lr = base_lr * decay
#         K.set_value(model.optimizer.lr, lr)
        history = model.fit(input_train_dict, y_train, 
                  batch_size=5000, 
                  epochs=1,
                  verbose=1,
                  shuffle=True,
                  )
        preds = model.predict(input_val_dict,5000,verbose=1)
        logloss = log_loss(y_val,preds)
        train_loss_list.append(history.history['loss'])
        val_loss_list.append(logloss)
        roc = roc_auc_score(y_val,preds)
        print(logloss)
        print(roc)
        if best_logloss is None:
            best_logloss = logloss
            best_weights = model.get_weights()
        else:
            if best_logloss > logloss:
                best_logloss = logloss
                best_weights = model.get_weights()
                cur_to = 0
            else:
                cur_to +=1
        if cur_to == tolerance:
            break
        print('best logloss is: {}'.format(best_logloss))
        print('remainning trial is: {}/{}'.format(cur_to,tolerance))
        print('total epoch trained: {}'.format(count))
        count+=1
    model.set_weights(best_weights)
    train_preds = model.predict(input_train_dict,5000,verbose=1)
    print('training loss is: {}'.format(log_loss(y_train,train_preds)))
    return model,train_loss_list,val_loss_list
    

In [26]:
model,t,v = train_each_fold(train_fold_dict[0],val_fold_dict[0],train_fold_y[0],val_fold_y[0],non_doc_col,doc_col,numu_col=numu_col,tolerance=35)

non doc ebd size:23
Epoch 1/1
137376/137376 [==============================] - 1s 7us/step
0.4198495430053561
0.7601145951060395
best logloss is: 0.4198495430053561
remainning trial is: 0/35
total epoch trained: 0
Epoch 1/1
137376/137376 [==============================] - 1s 5us/step
0.4179941665967334
0.7634190618376229
best logloss is: 0.4179941665967334
remainning trial is: 0/35
total epoch trained: 1
Epoch 1/1
137376/137376 [==============================] - 1s 5us/step
0.41784807073865976
0.7644252442172135
best logloss is: 0.41784807073865976
remainning trial is: 0/35
total epoch trained: 2
Epoch 1/1
137376/137376 [==============================] - 1s 5us/step
0.4176735252091553
0.7645694502850404
best logloss is: 0.4176735252091553
remainning trial is: 0/35
total epoch trained: 3
Epoch 1/1
137376/137376 [==============================] - 1s 5us/step
0.4176113794017469
0.7651177347906271
best logloss is: 0.4176113794017469
remainning trial is: 0/35
total epoch trained: 4
Epoch 1/

137376/137376 [==============================] - 1s 5us/step
0.41565861117556213
0.7690625732884226
best logloss is: 0.4150172980774767
remainning trial is: 3/35
total epoch trained: 28
Epoch 1/1
137376/137376 [==============================] - 1s 5us/step
0.4152901784192325
0.7689842899563202
best logloss is: 0.4150172980774767
remainning trial is: 4/35
total epoch trained: 29
Epoch 1/1
137376/137376 [==============================] - 1s 5us/step
0.41657607268614677
0.7688891027853362
best logloss is: 0.4150172980774767
remainning trial is: 5/35
total epoch trained: 30
Epoch 1/1
137376/137376 [==============================] - 1s 5us/step
0.41512871127400636
0.7691692644006127
best logloss is: 0.4150172980774767
remainning trial is: 6/35
total epoch trained: 31
Epoch 1/1
137376/137376 [==============================] - 1s 4us/step
0.41523566663724754
0.7692716603899412
best logloss is: 0.4150172980774767
remainning trial is: 7/35
total epoch trained: 32
Epoch 1/1
137376/137376 [======

137376/137376 [==============================] - 1s 5us/step
0.4149606333983077
0.7696404861492001
best logloss is: 0.41476793023364594
remainning trial is: 20/35
total epoch trained: 57
Epoch 1/1
137376/137376 [==============================] - 1s 5us/step
0.4154762307968536
0.7690830364010606
best logloss is: 0.41476793023364594
remainning trial is: 21/35
total epoch trained: 58
Epoch 1/1
137376/137376 [==============================] - 1s 5us/step
0.4152565033353288
0.7696640721987671
best logloss is: 0.41476793023364594
remainning trial is: 22/35
total epoch trained: 59
Epoch 1/1
137376/137376 [==============================] - 1s 5us/step
0.41503967230683186
0.7696433352701693
best logloss is: 0.41476793023364594
remainning trial is: 23/35
total epoch trained: 60
Epoch 1/1
137376/137376 [==============================] - 1s 5us/step
0.4150434302277937
0.7690294010105221
best logloss is: 0.41476793023364594
remainning trial is: 24/35
total epoch trained: 61
Epoch 1/1
137376/137376 

In [ ]:
model = get_nn_model(non_doc_col,doc_col,numu_cols=[])

In [ ]:
model.optimizer.lr.

In [ ]:
input_val_dict = val_fold_dict[0]
y_val = val_fold_y[0]
preds = model.predict(input_val_dict,5000,verbose=1)
logloss = log_loss(y_val,preds)
roc = roc_auc_score(y_val,preds)
print(logloss)
print(roc)

In [ ]:
# predsub = model.predict(test_input_dict,5000,verbose=1)
# test_save = test[['instance_id']].copy()
# test_save['predicted_score'] = predsub

In [ ]:
# test_save.to_csv('ebd_nn.csv',index=False)


In [ ]:
model = get_nn_model(non_doc_col,doc_col,numu_col)

In [ ]:
numu_col

In [ ]:
model.summary()

In [ ]:
# from keras.utils import plot_model
# plot_model(model, to_file='model.png')